This is a code to crop the eye region of images in the dataset for eye gaze model training

In [2]:
import os
import cv2
import shutil
import pandas as pd
from tqdm import tqdm

# Load the pre-trained Haar cascade classifier for eye detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')#_tree_eyeglasses.xml')
unprocessed_count = 0

def process_image(image_path, output_folder):
    global unprocessed_count
    # Load the image
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Failed to load image {image_path}")
        return None, False

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect eyes in the image
    eyes = eye_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(eyes) == 2:
        # Sort detected eyes by x coordinate to ensure left-to-right order
        eyes = sorted(eyes, key=lambda x: x[0])

        # Get the coordinates for the combined bounding box
        x1 = min(eyes[0][0], eyes[1][0])
        y1 = min(eyes[0][1], eyes[1][1])
        x2 = max(eyes[0][0] + eyes[0][2], eyes[1][0] + eyes[1][2])
        y2 = max(eyes[0][1] + eyes[0][3], eyes[1][1] + eyes[1][3])

        # Add some margin to the bounding box
        margin_y = int((y2 - y1) * 0.20)  # 20% of the height of the bounding box

        x1 = max(0, x1)
        y1 = max(0, y1 + margin_y)
        x2 = min(image.shape[1], x2)
        y2 = min(image.shape[0], y2 - margin_y)

        # Crop the region containing both eyes with margin
        eye_region = image[y1:y2, x1:x2]
        return eye_region, True
    else:
        unprocessed_count += 1
        return None, False

def process_folders(parent_folder):
    for subfolder in tqdm(os.listdir(parent_folder)):
        subfolder_path = os.path.join(parent_folder, subfolder)
        if os.path.isdir(subfolder_path):
            # Create the output directory
            output_folder = os.path.join('processed', os.path.basename(parent_folder), subfolder)
            os.makedirs(output_folder, exist_ok=True)

            # Initialize a DataFrame for valid CSV rows
            valid_csv_rows = pd.DataFrame()

            # Process images in the subfolder
            csv_file_path = None
            for file in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file)
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_name, _ = os.path.splitext(file)
                    eye_region, eyes_detected = process_image(file_path, output_folder)
                    if eyes_detected:
                        output_image_path = os.path.join(output_folder, f'{image_name}.png')
                        cv2.imwrite(output_image_path, eye_region)
                elif file.lower().endswith('.csv'):
                    csv_file_path = file_path
            
            # Process the CSV file
            if csv_file_path:
                df = pd.read_csv(csv_file_path)
                # Filter rows based on valid image names
                for file in os.listdir(subfolder_path):
                    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        image_name, _ = os.path.splitext(file)
                        if os.path.exists(os.path.join(output_folder, f'{image_name}.png')):
                            valid_csv_rows = pd.concat([valid_csv_rows, df[df['id'] == int(image_name)]])
                # Save the filtered CSV
                valid_csv_rows.to_csv(os.path.join(output_folder, 'lookat_points.csv'), index=False)
                print(f"Processed and filtered CSV saved to {output_folder}")

# Example usage
parent_folder = 'User_08'  # Replace with the path to your parent folder
process_folders(parent_folder)
print('Unprocessed Count: ', unprocessed_count)


  1%|▋                                                                                 | 1/125 [00:19<39:23, 19.06s/it]

Processed and filtered CSV saved to processed\User_08\HP_001


  2%|█▎                                                                                | 2/125 [00:35<36:11, 17.66s/it]

Processed and filtered CSV saved to processed\User_08\HP_002


  2%|█▉                                                                                | 3/125 [00:53<35:45, 17.59s/it]

...

Processed and filtered CSV saved to processed\User_08\HP_123


 99%|███████████████████████████████████████████████████████████████████████████████▎| 124/125 [34:03<00:14, 14.92s/it]

Processed and filtered CSV saved to processed\User_08\HP_124


100%|████████████████████████████████████████████████████████████████████████████████| 125/125 [34:18<00:00, 16.47s/it]

Processed and filtered CSV saved to processed\User_08\HP_125
Unprocessed Count:  183
